<a href="https://colab.research.google.com/github/chandler003/cecs_545_ai/blob/main/CSE_547_50_4242_Homework_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Deep Learning
# Algorithms and Methods

# Homework #02
# J. Chandler Hora

In [1]:
# STEP 1: Install Kaggle (only needed once)
#!pip install -q kaggle

# STEP 2: Upload your kaggle.json file (only needed once)
from google.colab import files
files.upload()  # Upload your kaggle.json here

# STEP 3: Move the key into the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# STEP 4: Download the Intel image classification dataset
!kaggle datasets download -d puneet6060/intel-image-classification

# STEP 5: Unzip the dataset
!unzip -q intel-image-classification.zip -d /content/intel-data

# STEP 6: Verify extracted folders
!ls /content/intel-data/seg_train


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
 74% 257M/346M [00:02<00:01, 88.5MB/s]
100% 346M/346M [00:02<00:00, 135MB/s] 
seg_train


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
#from tensorflow.keras.models import Model
#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
# ------------------------------
# GLOBAL PARAMETERS
# ------------------------------

IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32
NUM_CLASSES = 6
SEED = 42
EPOCHS = 10
KFOLD_SPLITS = 4
DATASET_DIR = '/content/intel-data'

# ------------------------------
# DATA LOADER (for manual batching)
# ------------------------------

def load_dataset_paths():
    """
    Scans the training directory and collects all image paths and class indices.
    Returns arrays of image paths and corresponding label indices.
    """
    train_dir = os.path.join(DATASET_DIR, 'seg_train', 'seg_train')
    class_names = sorted(os.listdir(train_dir))
    filepaths, labels = [], []

    for label_index, class_name in enumerate(class_names):
        class_dir = os.path.join(train_dir, class_name)
        for fname in os.listdir(class_dir):
            filepaths.append(os.path.join(class_dir, fname))
            labels.append(label_index)

    return np.array(filepaths), np.array(labels), class_names

def load_batch(image_paths):
    """
    Given a list of image file paths, load and preprocess them.
    """
    batch = []
    for path in image_paths:
        img = load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH))
        img_array = img_to_array(img) / 255.0
        batch.append(img_array)
    return np.array(batch)

# ------------------------------
# MODEL DEFINITIONS
# ------------------------------

def build_baseline_model():
    """
    Returns a simple CNN model with 3 Conv2D layers and 2 Dense layers.
    """
    model = models.Sequential()
    model.add(layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_complex_model():
    """
    Returns a deeper CNN model with additional Conv2D and Dense layers.
    """
    model = models.Sequential()
    model.add(layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ------------------------------
# TRAINING WITH K-FOLD CV
# ------------------------------

def train_kfold(model_builder, filepaths, labels):
    """
    Trains a given model builder using K-fold cross-validation.
    Returns a list of final validation accuracies for each fold.
    """
    kfold = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=SEED)
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(filepaths), 1):
        print(f"\n--- Fold {fold} ---")

        x_train_paths = filepaths[train_idx]
        x_val_paths = filepaths[val_idx]
        y_train = to_categorical(labels[train_idx], NUM_CLASSES)
        y_val = to_categorical(labels[val_idx], NUM_CLASSES)

        x_train = load_batch(x_train_paths)
        x_val = load_batch(x_val_paths)

        model = model_builder()
        history = model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            verbose=0
        )

        val_acc = history.history['val_accuracy'][-1]
        print(f"Validation Accuracy: {val_acc:.4f}")
        fold_accuracies.append(val_acc)

    return fold_accuracies

# ------------------------------
# MAIN WORKFLOW
# ------------------------------

def main():
    filepaths, labels, class_names = load_dataset_paths()

    print(f"Classes: {class_names}")
    print(f"Total Samples: {len(filepaths)}")

    baseline_scores = train_kfold(build_baseline_model, filepaths, labels)
    complex_scores = train_kfold(build_complex_model, filepaths, labels)

    # Display cross-validation results
    df = pd.DataFrame({
        "Fold": list(range(1, KFOLD_SPLITS + 1)),
        "Baseline CNN Accuracy": baseline_scores,
        "Complex CNN Accuracy": complex_scores
    })

    print("\nCross-Validation Accuracy Summary:")
    print(df)
    print("\nAverage Accuracy:")
    print(df.mean(numeric_only=True))

if __name__ == "__main__":
    main()


Classes: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
Total Samples: 14034

--- Fold 1 ---
Validation Accuracy: 0.7854

--- Fold 2 ---
Validation Accuracy: 0.7558

--- Fold 3 ---
Validation Accuracy: 0.7873

--- Fold 4 ---
Validation Accuracy: 0.7597

--- Fold 1 ---
Validation Accuracy: 0.7771

--- Fold 2 ---
Validation Accuracy: 0.7769

--- Fold 3 ---
Validation Accuracy: 0.7768

--- Fold 4 ---
Validation Accuracy: 0.7563

Cross-Validation Accuracy Summary:
   Fold  Baseline CNN Accuracy  Complex CNN Accuracy
0     1               0.785409              0.777144
1     2               0.755771              0.776860
2     3               0.787343              0.776796
3     4               0.759692              0.756271

Average Accuracy:
Fold                     2.500000
Baseline CNN Accuracy    0.772054
Complex CNN Accuracy     0.771768
dtype: float64
